In [ ]:
#import packages

from skimage.io import imread, imsave
from skimage.transform import rescale
from skimage.exposure import rescale_intensity
from shapely.geometry import  MultiPolygon, Polygon
from collections import defaultdict
import geopandas
import pandas as pd
import cv2
import numpy as np
import json
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#import CCFv3 template reference lists of all experimental groups

WT_ctrl_list = pd.read_csv("WT_ctrl_list.csv", index_col = 0)
Tbr1_ctrl_list= pd.read_csv("Tbr1_ctrl_list.csv", index_col = 0)
WT_TBS_list= pd.read_csv("WT_TBS_list.csv", index_col = 0)
Tbr1_TBS_list= pd.read_csv("Tbr1_TBS_list.csv", index_col = 0)

reference_summary = pd.DataFrame ([WT_ctrl_list.columns, Tbr1_ctrl_list.columns, WT_TBS_list.columns, Tbr1_TBS_list.columns])

pd.DataFrame(reference_summary.T.values, columns = ["WT_ctrl", "Tbr1_ctrl", "WT_TBS", "Tbr1_TBS"])

In [ ]:
#create regional masks

structure_list_dropempty = pd.read_csv ("./CCFv3/structuretree/structure_list_paper.csv", index_col = 0)

mask_ref = pd.concat([structure_list_dropempty.IDX, structure_list_dropempty.LABEL], axis=1, sort=False)

def structure_mask_L (genotype, st_pattern, reference_list, structure_list):
    """create left brain regional mask, genotype in str, reference_list in pd.Dataframe, structure_list in pd.DataFrame.columns"""
    for a in structure_list:
        structure_index = int (structure_list_dropempty[structure_list_dropempty.LABEL ==a].index.values)
        for b in reference_list.columns:
            #create folder for regional masks storage (demo purpose)
            #demo data provide real ROI corrected regional masks
            structure_path = ("./quantification/"+ genotype+"/"+st_pattern +"/mask_L/"+a+"/"+b)
            if not os.path.exists(structure_path):
                os.makedirs(structure_path)

            for c in reference_list[b].dropna():
                if int (c) in json.loads(structure_list_dropempty["section_number"][structure_index]):
                    #CCFv3 coronal masks are downloaded through AllenSDK https://allensdk.readthedocs.io/en/latest/_static/examples/nb/reference_space.html
                    mask_l = pd.read_csv ("./CCFv3/mask/mask"+str(int(mask_ref.IDX[mask_ref.LABEL ==a]))+"/mask"+str(int(c))+".csv", header = 0, index_col = 0).values[:, :570].astype("uint8")
                    mask_l = rescale_intensity (mask_l)
                    imsave (structure_path+"/"+"%04d"%(int(reference_list[b][reference_list[b]==c].index.values)+1)+".png", mask_l)

def structure_mask_R (genotype, st_pattern, reference_list, structure_list):
    """create left brain regional mask, genotype in str, reference_list in pd.Dataframe, structure_list in list"""
    for d in structure_list:
        structure_index = int (structure_list_dropempty[structure_list_dropempty.LABEL ==d].index.values)
        for e in reference_list.columns:
            #create folder for regional masks storage (demo purpose)
            #demo data provide real ROI corrected regional masks
            structure_path = ("./quantification/"+ genotype+"/"+st_pattern +"/mask_R/"+d+"/"+e)
            if not os.path.exists(structure_path):
                os.makedirs(structure_path)

            for f in reference_list[e].dropna():
                if int (f) in json.loads(structure_list_dropempty["section_number"][structure_index]):
                    #CCFv3 coronal masks are downloaded through AllenSDK https://allensdk.readthedocs.io/en/latest/_static/examples/nb/reference_space.html
                    mask_R = pd.read_csv ("./CCFv3/mask/mask"+str(int(mask_ref.IDX[mask_ref.LABEL ==d]))+"/mask"+str(int(f))+".csv", header = 0, index_col = 0).values[:, 570:].astype("uint8")
                    mask_R = rescale_intensity (mask_R)
                    imsave (structure_path+"/"+"%04d"%(int(reference_list[e][reference_list[e]==f].index.values)+1)+".png", mask_R)

In [ ]:
#create structure mask
DMN_list = ["ACAd", "ACAv", "PL", "ILA", "ORBl", "ORBm", "ORBvl", 
            "VISa", "VISam", "RSPagl", "RSPd", "RSPv", 
            "SSp-tr", "SSp-ll", "MOs"]

structure_mask_L ("WT", "ctrl", WT_ctrl_list, DMN_list)
structure_mask_L ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
structure_mask_L ("WT", "TBS", WT_TBS_list, DMN_list)
structure_mask_L ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

structure_mask_R ("WT", "ctrl", WT_ctrl_list, DMN_list)
structure_mask_R ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
structure_mask_R ("WT", "TBS", WT_TBS_list, DMN_list)
structure_mask_R ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)


#convert regional masks(.png) to Image J ROI(RoiSet.zip) by **createROI_all.ijm**  

In [ ]:
# delete old regional masks(.png)

def remove_mask(genotype, st_pattern, mask_hemisphere, reference_list, structure_list):
    """remove mask_tif used for ROI correction"""    
    for g in structure_list:
        for h in reference_list.columns:
            structure_path = ("./quantification/"+genotype+"/"+st_pattern+"/"+mask_hemisphere+"/"+g+"/"+h)
            for root, dirs, files in os.walk (structure_path):
                for file_l in files:
                    if file_l.endswith (".png"):
                        os.remove (structure_path+"/"+file_l)
                        
DMN_list = ["ACAd", "ACAv", "PL", "ILA", "ORBl", "ORBm", "ORBvl", 
            "VISa", "VISam", "RSPagl", "RSPd", "RSPv", 
            "SSp-tr", "SSp-ll", "MOs"]

remove_mask ("WT", "ctrl", "mask_L", WT_ctrl_list, DMN_list)
remove_mask ("Tbr1", "ctrl", "mask_L", Tbr1_ctrl_list, DMN_list)
remove_mask ("WT", "TBS", "mask_L", WT_TBS_list, DMN_list)
remove_mask ("Tbr1", "TBS", "mask_L", Tbr1_TBS_list, DMN_list)

remove_mask ("WT", "ctrl", "mask_R", WT_ctrl_list, DMN_list)
remove_mask ("Tbr1", "ctrl", "mask_R", Tbr1_ctrl_list, DMN_list)
remove_mask ("WT", "TBS", "mask_R", WT_TBS_list, DMN_list)
remove_mask ("Tbr1", "TBS", "mask_R", Tbr1_TBS_list, DMN_list)

#load reference image stack, other reference ROI, ROI Manager by  
  **open_L_image_stack_ROI_correction.ijm** or **open_R_image_stack_ROI_correction.ijm**  
#manually correct regional ROI by Image J tools, save updated RoiSet.zip  
#create updated regional masks(.png), roi_list.csv by **convertROI_into_mask.py**

In [ ]:
#rename mask function

def rename_mask_L (genotype, st_pattern ,reference_list, structure_list):
    """rename left brain corrected mask name to allen brain regional mask name"""
    for g in structure_list:
        
        for h in reference_list.columns:
            structure_path = ("./quantification/"+ genotype+ "/"+ st_pattern +"/mask_L/"+g+"/"+h)
            str_col = {"Name":"str"}
            roi_list = pd.read_csv (structure_path+"/roi_list.csv", index_col = 0, dtype = str_col)
            
            print (roi_list)
            for indexi, varai in roi_list.iterrows():
                os.rename(structure_path+"/mask"+(roi_list["Name"][indexi])+".png", structure_path+"/mask"+str(int(reference_list[h][roi_list["Z"][indexi]-1]))+".png")
                
def rename_mask_R (genotype, st_pattern, reference_list, structure_list):
    """rename right brain corrected mask name to allen brain regional mask name"""
    for g in structure_list:
        
        for h in reference_list.columns:
            structure_path = ("./quantification/"+ genotype+"/"+ st_pattern +"/mask_R/"+g+"/"+h)
            str_col = {"Name":"str"}
            roi_list = pd.read_csv (structure_path+"/roi_list.csv", index_col = 0, dtype = str_col)
            
            print (roi_list)
            for indexi, varai in roi_list.iterrows():
                os.rename(structure_path+"/mask"+(roi_list["Name"][indexi])+".png", structure_path+"/mask"+str(int(reference_list[h][roi_list["Z"][indexi]-1]))+".png")
                

In [ ]:
#rename mask
DMN_list = ["ACAd", "ACAv", "PL", "ILA", "ORBl", "ORBm", "ORBvl", 
            "VISa", "VISam", "RSPagl", "RSPd", "RSPv", 
            "SSp-tr", "SSp-ll", "MOs"]

rename_mask_L ("WT", "ctrl", WT_ctrl_list, DMN_list)
rename_mask_L ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
rename_mask_L ("WT", "TBS", WT_TBS_list, DMN_list)
rename_mask_L ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

rename_mask_R ("WT", "ctrl", WT_ctrl_list, DMN_list)
rename_mask_R ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
rename_mask_R ("WT", "TBS", WT_TBS_list, DMN_list)
rename_mask_R ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

In [ ]:
#re quantification by corrected ROI

#mask to polygons function
#code origin: https://michhar.github.io/masks_to_polygons_and_back/
def mask_to_polygons(mask, epsilon=10., min_area=10.):
    """Convert a mask ndarray (binarized image) to Multipolygons"""
    # first, find contours with cv2: it's much faster than shapely
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    if not contours:
        return MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    all_polygons = MultiPolygon(all_polygons)

    return all_polygons

In [ ]:
#reanalysis based on new regional mask function

def reanalysis_oChIEF_cfos_L (genotype, st_pattern, reference_list, structure_list):
    """reanalysis left brain oChIEF and cfos signals based on new regional mask"""
    for g in structure_list:
    
        for h in reference_list.columns:
            structure_path = ("./quantification/"+genotype+"/"+st_pattern +"/"+h+"/requantification/"+g)
            if not os.path.exists(structure_path):
                os.makedirs(structure_path)
            mask_path = ("./quantification/"+ genotype+"/"+st_pattern +"/mask_L/"+g+"/"+h)
            roi_list = pd.read_csv (mask_path+"/roi_list.csv", index_col = 0)
            
            cfos_spot_L = []
            spot_L_size = []
            area_all_L =[]
            cfos_L_densitylist =[]
            oChIEF_L_binary_list = []
            slice_number = []
            
            for indexi, varai in reference_list.iterrows():
                if indexi +1 in roi_list["Z"].values:
                    slice_n = indexi+1
                    slice_number.append(slice_n)

                    mask_L = imread (mask_path+"/mask"+str (int(reference_list[h][indexi]))+".png")
                    #scale mask from 720 x 800 to 3240 x 4547 
                    highcontent_mask = rescale (mask_L, 5.6842105263, anti_aliasing=False)
                    highcontent_mask_bw =(highcontent_mask>np.mean (highcontent_mask)).astype("uint8")
                    mask_poly = mask_to_polygons(highcontent_mask_bw, min_area = 1)
                    mask_geopd = geopandas.GeoSeries([mask_poly])
                    spot = pd.read_csv ("./"+h+"/cfos_spot/spot"+str(indexi+1)+".csv",  header = 0, index_col = 0)
                    
                    spotL = spot[spot["0"]<3240]                
                    spotL_geopd = geopandas.GeoDataFrame (spotL, geometry = geopandas.points_from_xy(spotL["0"], spotL["1"]))
                    cfosL_density = float(len(spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))])/mask_geopd.area)/(1.76**2)*1000000
                    cfos_L_densitylist.append (cfosL_density)
                    cfosL_spot = np.transpose([spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))]["0"].tolist(), 
                                               spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))]["1"].tolist(),
                                               (np.zeros (len (spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))]["0"]))+slice_n).tolist()]).tolist()
                    cfos_spot_L+=cfosL_spot
                    spot_L_size.append (len (spotL_geopd[spotL_geopd.within(mask_poly.buffer(0))]))
                    area_all_L.append (float(mask_geopd.area))
                    
                    binary_img = imread("./"+h+"/oChIEF_T_b/oChIEF_T_b"+"%04d"%indexi+".png")[:,:3240]
                    L_b = np.where (highcontent_mask>np.mean(highcontent_mask), binary_img, 0)
                    mean_L_b = (np.mean (L_b))
                    oChIEF_L_binary_list.append(mean_L_b)

            pd.DataFrame (list(zip(slice_number, cfos_L_densitylist)),columns=["slice_number","cfos_L_density"]).to_csv (structure_path +"/cfos_density_L.csv")
            pd.DataFrame ({"slice_number":slice_number, "cfos_L_spot_number": spot_L_size, "structure_area (pixels)": area_all_L}).to_csv (structure_path +"/cfos_density_L_raw.csv")
            pd.DataFrame (cfos_spot_L, columns = ["x (pixels)","y(pixels)","z(pixels)"]).to_csv (structure_path +"/cfos_L_spot_location.csv")            
            pd.DataFrame (list(zip(slice_number, oChIEF_L_binary_list)),columns=["slice_number","oChIEF_L_b_mean"]).to_csv (structure_path +"/oChIEF_meanint_L_b.csv")

def reanalysis_oChIEF_cfos_R (genotype, st_pattern, reference_list, structure_list):
    """reanalysis right brain oChIEF and cfos signals based on new regional mask"""
    for g in structure_list:
    
        for h in reference_list.columns:
            structure_path = ("./quantification/"+genotype+"/"+st_pattern+"/"+h+"/requantification/"+g)
            if not os.path.exists(structure_path):
                os.makedirs(structure_path)
            mask_path = ("./quantification/"+ genotype+"/"+st_pattern+ "/mask_R/"+g+"/"+h)
            roi_list = pd.read_csv (mask_path+"/roi_list.csv", index_col = 0)
            
            cfos_spot_R = []
            spot_R_size = []
            area_all_R =[]
            cfos_R_densitylist =[]
            oChIEF_R_binary_list = []
            slice_number = []
            
            for indexi, varai in reference_list.iterrows():
                if indexi +1 in roi_list["Z"].values:
                    slice_n = indexi+1
                    slice_number.append(slice_n)

                    mask_R = imread (mask_path+"/mask"+str (int(reference_list[h][indexi]))+".png")
                    #scale mask from 720 x 800 to 3240 x 4547 
                    highcontent_mask = rescale (mask_R, 5.6842105263, anti_aliasing=False)
                    highcontent_mask_bw =(highcontent_mask>np.mean (highcontent_mask)).astype("uint8")
                    mask_poly = mask_to_polygons(highcontent_mask_bw, min_area = 1)
                    mask_geopd = geopandas.GeoSeries([mask_poly])
                    spot = pd.read_csv ("./"+h+"/cfos_spot/spot"+str(indexi+1)+".csv",  header = 0, index_col = 0)
                    
                    spot_R = spot[spot["0"]>=3240]
                    spotR = pd.DataFrame()
                    spotR.loc[:,"0"]= spot_R.loc[:,"0"]-3240
                    spotR.loc[:,"1"]= spot_R.loc[:,"1"]
                    spotR_geopd = geopandas.GeoDataFrame (spotR, geometry = geopandas.points_from_xy(spotR["0"], spotR["1"]))
                    cfosR_density = float(len(spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))])/mask_geopd.area)/(1.76**2)*1000000
                    cfos_R_densitylist.append (cfosR_density)
                    cfosR_spot = np.transpose([spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))]["0"].tolist(), 
                                               spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))]["1"].tolist(), 
                                               (np.zeros (len (spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))]["0"]))+slice_n).tolist()]).tolist()
                    cfos_spot_R+=cfosR_spot
                    spot_R_size.append (len (spotR_geopd[spotR_geopd.within(mask_poly.buffer(0))]))
                    area_all_R.append (float(mask_geopd.area))
                    
                    binary_img = imread("./"+h+"/oChIEF_T_b/oChIEF_T_b"+"%04d"%indexi+".tif")[:,3240:]
                    L_b = np.where (highcontent_mask>np.mean(highcontent_mask), binary_img, 0)
                    mean_L_b = (np.mean (L_b))
                    oChIEF_R_binary_list.append(mean_L_b)

            pd.DataFrame (list(zip(slice_number, cfos_R_densitylist)),columns=["slice_number","cfos_R_density"]).to_csv (structure_path +"/cfos_density_R.csv")
            pd.DataFrame ({"slice_number":slice_number, "cfos_R_spot_number": spot_R_size, "structure_area (pixels)": area_all_R}).to_csv (structure_path +"/cfos_density_R_raw.csv")
            pd.DataFrame (cfos_spot_R, columns = ["x (pixels)","y(pixels)","z(pixels)"]).to_csv (structure_path +"/cfos_R_spot_location.csv")
            pd.DataFrame (list(zip(slice_number, oChIEF_R_binary_list)),columns=["slice_number","oChIEF_R_b_mean"]).to_csv (structure_path +"/oChIEF_meanint_R_b.csv")

In [ ]:
#reanalysis based on new regional mask
DMN_list = ["ACAd", "ACAv", "PL", "ILA", "ORBl", "ORBm", "ORBvl", 
            "VISa", "VISam", "RSPagl", "RSPd", "RSPv", 
            "SSp-tr", "SSp-ll", "MOs"]

reanalysis_oChIEF_cfos_L ("WT", "ctrl", WT_ctrl_list, DMN_list)
reanalysis_oChIEF_cfos_L ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
reanalysis_oChIEF_cfos_L ("WT", "TBS", WT_TBS_list, DMN_list)
reanalysis_oChIEF_cfos_L ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

reanalysis_oChIEF_cfos_R ("WT", "ctrl", WT_ctrl_list, DMN_list)
reanalysis_oChIEF_cfos_R ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
reanalysis_oChIEF_cfos_R ("WT", "TBS", WT_TBS_list, DMN_list)
reanalysis_oChIEF_cfos_R ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

In [ ]:
#rewrite the data of oChIEF_mean_b and C-FOS_cell_density

def rewrite_oChIEF_cfos_density_summary_L (genotype, st_pattern, reference_list, structure_list):
    """rewrite the data of oChIEF and cfos data in left brain"""
    
    for h in reference_list.columns:
        
        oChIEF_mean_L_b = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/oChIEF_L_summary_b.csv", index_col = 0)
        cfos_density_L = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/cfos_L_summary.csv", index_col = 0)
        
        for g in structure_list:
            
            re_oChIEF_L_b = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern +"/"+h+"/requantification/"+g+"/oChIEF_meanint_L_b.csv", index_col = 0)
            re_cfos_L_density = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern +"/"+h+"/requantification/"+g+"/cfos_density_L.csv", index_col = 0)

            oChIEF_mean_L_b.loc[g]=np.nan
            cfos_density_L.loc[g]=np.nan

            for indexa, varaa in re_cfos_L_density.iterrows():
                oChIEF_mean_L_b.loc[g][re_oChIEF_L_b.loc[indexa, "slice_number"]-1] = (re_oChIEF_L_b.loc[indexa, "oChIEF_L_b_mean"])/255
                cfos_density_L.loc[g][re_cfos_L_density.loc[indexa, "slice_number"]-1] = re_cfos_L_density.loc[indexa, "cfos_L_density"]

        oChIEF_mean_L_b.to_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/rre_oChIEF_L_summary_b.csv")
        cfos_density_L.to_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/rre_cfos_L_summary.csv")
            
def rewrite_oChIEF_cfos_density_summary_R (genotype, st_pattern, reference_list, structure_list):
    """rewrite the data of oChIEF and cfos data in right brain"""
    
    for h in reference_list.columns:
        
        oChIEF_mean_R_b = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/oChIEF_R_summary_b.csv", index_col = 0)
        cfos_density_R = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/cfos_R_summary.csv", index_col = 0)
        
        for g in structure_list:
            
            re_oChIEF_R_b = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern +"/"+h+"/requantification/"+g+"/oChIEF_meanint_R_b.csv", index_col = 0)
            re_cfos_R_density = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern +"/"+h+"/requantification/"+g+"/cfos_density_R.csv", index_col = 0)

            oChIEF_mean_R_b.loc[g]=np.nan
            cfos_density_R.loc[g]=np.nan

            for indexa, varaa in re_cfos_R_density.iterrows():                
                oChIEF_mean_R_b.loc[g][re_oChIEF_R_b.loc[indexa, "slice_number"]-1] = (re_oChIEF_R_b.loc[indexa, "oChIEF_R_b_mean"])/255
                cfos_density_R.loc[g][re_cfos_R_density.loc[indexa, "slice_number"]-1] = re_cfos_R_density.loc[indexa, "cfos_R_density"]

        oChIEF_mean__R_b.to_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/rre_oChIEF_R_summary_b.csv")
        cfos_density_R.to_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/rre_cfos_R_summary.csv")

In [ ]:
#rewrite the data of oChIEF_mean_b and C-FOS_cell_density

DMN_list = ["ACAd", "ACAv", "PL", "ILA", "ORBl", "ORBm", "ORBvl", 
            "VISa", "VISam", "RSPagl", "RSPd", "RSPv", 
            "SSp-tr", "SSp-ll", "MOs"]

rewrite_oChIEF_cfos_density_summary_L ("WT", "ctrl", WT_ctrl_list, DMN_list)
rewrite_oChIEF_cfos_density_summary_L ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
rewrite_oChIEF_cfos_density_summary_L ("WT", "TBS", WT_TBS_list, DMN_list)
rewrite_oChIEF_cfos_density_summary_L ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

rewrite_oChIEF_cfos_density_summary_R ("WT", "ctrl", WT_ctrl_list, DMN_list)
rewrite_oChIEF_cfos_density_summary_R ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
rewrite_oChIEF_cfos_density_summary_R ("WT", "TBS", WT_TBS_list, DMN_list)
rewrite_oChIEF_cfos_density_summary_R ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

In [ ]:
#re-write cfos_number_summary

def rewrite_cfos_num_summary_L (genotype, st_pattern, reference_list, structure_list):
    """rewrite the data of cfos_num_summary in left brain"""
    
    for h in reference_list.columns:
        cfos_L = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/cfos_L_num_summary.csv", index_col = 0)
                
        for g in structure_list:
            re_YFP_cell_density_raw_L = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/requantification/"+g+"/cfos_density_L_raw.csv", index_col = 0)            
            cfos_L.loc[g]=np.nan

            for indexa, varaa in re_YFP_cell_density_raw_L.iterrows():
                cfos_L.loc[g][re_YFP_cell_density_raw_L.loc[indexa, "slice_number"]-1] = (re_YFP_cell_density_raw_L.loc[indexa, "cfos_L_spot_number"])

        cfos_L.to_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/rre_cfos_L_num_summary.csv")
        
def rewrite_cfos_num_summary_R (genotype, st_pattern, reference_list, structure_list):
    """rewrite the data of cfos_num_summary in right brain"""
    
    for h in reference_list.columns:
        cfos_R = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/cfos_R_num_summary.csv", index_col = 0)
                
        for g in structure_list:
            re_YFP_cell_density_raw_R = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/requantification/"+g+"/cfos_density_R_raw.csv", index_col = 0)            
            cfos_R.loc[g]=np.nan

            for indexa, varaa in re_YFP_cell_density_raw_R.iterrows():
                cfos_R.loc[g][re_YFP_cell_density_raw_R.loc[indexa, "slice_number"]-1] = (re_YFP_cell_density_raw_R.loc[indexa, "cfos_R_spot_number"])

        cfos_R.to_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/rre_cfos_R_num_summary.csv")

In [ ]:
#re-write cfos_number_summary

DMN_list = ["ACAd", "ACAv", "PL", "ILA", "ORBl", "ORBm", "ORBvl", 
            "VISa", "VISam", "RSPagl", "RSPd", "RSPv", 
            "SSp-tr", "SSp-ll", "MOs"]

rewrite_cfos_num_summary_L ("WT", "ctrl", WT_ctrl_list, DMN_list)
rewrite_cfos_num_summary_L ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
rewrite_cfos_num_summary_L ("WT", "TBS", WT_TBS_list, DMN_list)
rewrite_cfos_num_summary_L ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

rewrite_cfos_num_summary_R ("WT", "ctrl", WT_ctrl_list, DMN_list)
rewrite_cfos_num_summary_R ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
rewrite_cfos_num_summary_R ("WT", "TBS", WT_TBS_list, DMN_list)
rewrite_cfos_num_summary_R ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

In [ ]:
#re-write area_summary

def rewrite_area_summary_L (genotype, st_pattern, reference_list, structure_list):
    """rewrite the data of area_summary in left brain"""
    
    for h in reference_list.columns:
        area_L = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/area_summary.csv", index_col = 0)
                
        for g in structure_list:
            re_YFP_cell_density_raw_L = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/requantification/"+g+"/cfos_density_L_raw.csv", index_col = 0)            
            area_L.loc[g]=np.nan

            for indexa, varaa in re_YFP_cell_density_raw_L.iterrows():
                area_L.loc[g][re_YFP_cell_density_raw_L.loc[indexa, "slice_number"]-1] = (re_YFP_cell_density_raw_L.loc[indexa, "structure_area (pixels)"])

        area_L.to_csv ("/Users/tsantinghsu/Desktop/BLA_axon_tracing/quantification/"+ genotype+"/"+st_pattern+"/"+h+"/rre_area_L_summary.csv")
        
def rewrite_area_summary_R (genotype, st_pattern, reference_list, structure_list):
    """rewrite the data of area_summary in right brain"""
    
    for h in reference_list.columns:
        area_R = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/area_summary.csv", index_col = 0)
                
        for g in structure_list:
            re_YFP_cell_density_raw_R = pd.read_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/requantification/"+g+"/cfos_density_R_raw.csv", index_col = 0)            
            area_R.loc[g]=np.nan

            for indexa, varaa in re_YFP_cell_density_raw_R.iterrows():
                area_R.loc[g][re_YFP_cell_density_raw_R.loc[indexa, "slice_number"]-1] = (re_YFP_cell_density_raw_R.loc[indexa, "structure_area (pixels)"])

        area_R.to_csv ("./quantification/"+ genotype+"/"+st_pattern+"/"+h+"/rre_area_R_summary.csv")

In [ ]:
#re-write area_summary

DMN_list = ["ACAd", "ACAv", "PL", "ILA", "ORBl", "ORBm", "ORBvl", 
            "VISa", "VISam", "RSPagl", "RSPd", "RSPv", 
            "SSp-tr", "SSp-ll", "MOs"]

rewrite_area_summary_L ("WT", "ctrl", WT_ctrl_list, DMN_list)
rewrite_area_summary_L ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
rewrite_area_summary_L ("WT", "TBS", WT_TBS_list, DMN_list)
rewrite_area_summary_L ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)

rewrite_area_summary_R ("WT", "ctrl", WT_ctrl_list, DMN_list)
rewrite_area_summary_R ("Tbr1", "ctrl", Tbr1_ctrl_list, DMN_list)
rewrite_area_summary_R ("WT", "TBS", WT_TBS_list, DMN_list)
rewrite_area_summary_R ("Tbr1", "TBS", Tbr1_TBS_list, DMN_list)
